In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import nibabel as nib
import matplotlib.pyplot as plt
from pathlib import *
from pylabs.utils.paths import getnetworkdataroot
fs = Path(getnetworkdataroot())
pd.set_option('display.width', 999999)
pd.options.display.max_colwidth = 250
pd.options.display.max_rows = 999

In [2]:
vtkf = fs / 'bbc' / 'allvtk_motor_and_cc_combined_dec14_2016.csv'
vtk_fbs = pd.read_csv(vtkf)

In [3]:
#this renames col label 'subjectid' with 'vtk_file_name'
vtk_fbs.rename(columns={'subjectid': 'vtk_file_name', 'icountgood': 'FB_reach_dest', 'icounttotal': 'total_num_FB'}, inplace=True)
#makes subj id fm 1st 10 chars of vtk_file_name
vtk_fbs['Subjid'] = vtk_fbs['vtk_file_name'].str[0:10]
#makes fit methods
vtk_fbs.loc[vtk_fbs['vtk_file_name'].str.contains('restore_cam_tensor_medfilt'), 'fit_method'] = 'restore_camino_mf'
vtk_fbs.loc[vtk_fbs['vtk_file_name'].str.contains('restore_cam_tensor_mori'), 'fit_method'] = 'restore_camino'
vtk_fbs.loc[vtk_fbs['vtk_file_name'].str.contains('wls_cam_tensor_medfilt'), 'fit_method'] = 'wls_camino_mf'
vtk_fbs.loc[vtk_fbs['vtk_file_name'].str.contains('wls_cam_tensor_mori'), 'fit_method'] = 'wls_camino'
vtk_fbs.loc[vtk_fbs['vtk_file_name'].str.contains('ols_cam_tensor_medfilt'), 'fit_method'] = 'ols_camino_mf'
vtk_fbs.loc[vtk_fbs['vtk_file_name'].str.contains('ols_cam_tensor_mori'), 'fit_method'] = 'ols_camino'
vtk_fbs.loc[vtk_fbs['vtk_file_name'].str.contains('ols_dipy_tensor_mori'), 'fit_method'] = 'ols_dipy'
vtk_fbs.loc[vtk_fbs['vtk_file_name'].str.contains('ols_dipy_tensor_med'), 'fit_method'] = 'ols_dipy_mf'
vtk_fbs.loc[vtk_fbs['vtk_file_name'].str.contains('wls_dipy_tensor_mori'), 'fit_method'] = 'wls_dipy'
vtk_fbs.loc[vtk_fbs['vtk_file_name'].str.contains('wls_dipy_tensor_medfilt'), 'fit_method'] = 'wls_dipy_mf'
vtk_fbs.loc[vtk_fbs['vtk_file_name'].str.contains('ols_fsl_tensor_medfilt'), 'fit_method'] = 'ols_fsl_mf'
vtk_fbs.loc[vtk_fbs['vtk_file_name'].str.contains('ols_fsl_tensor_mori'), 'fit_method'] = 'ols_fsl'
vtk_fbs.loc[vtk_fbs['vtk_file_name'].str.contains('wls_fsl_tensor_medfilt'), 'fit_method'] = 'wls_fsl_mf'
vtk_fbs.loc[vtk_fbs['vtk_file_name'].str.contains('wls_fsl_tensor_mori'), 'fit_method'] = 'wls_fsl'
#makes region names for regions col.
vtk_fbs.loc[vtk_fbs['vtk_file_name'].str.contains('LeftPostIntCap'), 'region'] = 'LeftPostIntCap'
vtk_fbs.loc[vtk_fbs['vtk_file_name'].str.contains('RightPostIntCap'), 'region'] = 'RightPostIntCap'
vtk_fbs.loc[vtk_fbs['vtk_file_name'].str.contains('Left_IFOF-70'), 'region'] = 'Left_IFOF'
vtk_fbs.loc[vtk_fbs['vtk_file_name'].str.contains('Right_IFOF-158'), 'region'] = 'Right_IFOF'
vtk_fbs.loc[vtk_fbs['vtk_file_name'].str.contains('mori_CC'), 'region'] = 'Corpus_Callosum'
#calculate %FB arrive at target as float and formatted text for display
vtk_fbs['%arrived'] = vtk_fbs['total_num_FB'].sub(vtk_fbs['FB_reach_dest']).div(vtk_fbs['total_num_FB'])
vtk_fbs['FB_percent_reach_dest'] = pd.Series(["{0:.2f}%".format(val*100) for val in vtk_fbs['%arrived']], index = vtk_fbs.index)

In [4]:
vtk_fbs.ix[31:90,['Subjid', 'fit_method', 'region', 'vtk_file_name', 'FB_reach_dest', 'total_num_FB', 'FB_percent_reach_dest']]

,Subjid,fit_method,region,vtk_file_name,FB_reach_dest,total_num_FB,FB_percent_reach_dest
31,sub-bbc105,NaN,RightPostIntCap,sub-bbc105_ses-1_dti_15dir_b1000_1_eddy_corrected_repol_std2_restore_cam2fsl_tensor_mori_RightPostIntCap-123.vtk,225,384,41.41%
32,sub-bbc105,wls_dipy_mf,LeftPostIntCap,sub-bbc105_ses-1_dti_15dir_b1000_1_eddy_corrected_repol_std2_wls_dipy_tensor_medfilt_mori_LeftPostIntCap-35.vtk,203,418,51.44%
33,sub-bbc105,wls_dipy_mf,RightPostIntCap,sub-bbc105_ses-1_dti_15dir_b1000_1_eddy_corrected_repol_std2_wls_dipy_tensor_medfilt_mori_RightPostIntCap-123.vtk,355,397,10.58%
34,sub-bbc105,wls_dipy,LeftPostIntCap,sub-bbc105_ses-1_dti_15dir_b1000_1_eddy_corrected_repol_std2_wls_dipy_tensor_mori_LeftPostIntCap-35.vtk,126,408,69.12%
35,sub-bbc105,wls_dipy,RightPostIntCap,sub-bbc105_ses-1_dti_15dir_b1000_1_eddy_corrected_repol_std2_wls_dipy_tensor_mori_RightPostIntCap-123.vtk,239,385,37.92%
36,sub-bbc105,wls_fsl_mf,LeftPostIntCap,sub-bbc105_ses-1_dti_15dir_b1000_1_eddy_corrected_repol_std2_wls_fsl_tensor_medfilt_mori_LeftPostIntCap-35.vtk,203,415,51.08%
37,sub-bbc105,wls_fsl_mf,RightPostIntCap,sub-bbc105_ses-1_dti_15dir_b1000_1_eddy_corrected_repol_std2_wls_fsl_tensor_medfilt_mori_RightPostIntCap-123.vtk,349,395,11.65%
38,sub-bbc105,wls_fsl,LeftPostIntCap,sub-bbc105_ses-1_dti_15dir_b1000_1_eddy_corrected_repol_std2_wls_fsl_tensor_mori_LeftPostIntCap-35.vtk,118,406,70.94%
39,sub-bbc105,wls_fsl,RightPostIntCap,sub-bbc105_ses-1_dti_15dir_b1000_1_eddy_corrected_repol_std2_wls_fsl_tensor_mori_RightPostIntCap-123.vtk,232,378,38.62%
40,sub-bbc106,ols_dipy_mf,LeftPostIntCap,sub-bbc106_ses-1_dti_15dir_b1000_1_eddy_corrected_repol_std2_ols_dipy_tensor_medfilt_mori_LeftPostIntCap-35.vtk,248,458,45.85%


In [10]:
sid_lst = vtk_fbs.Subjid.unique()
fitm_lst = vtk_fbs.fit_method.unique()
region_lst = vtk_fbs.region.unique()
idx_arr = [sid_lst, fitm_lst, region_lst]
tuples = list(zip(sid_lst, fitm_lst, region_lst))
vtk_fbs.index(index=idx_arr)

TypeError: 'RangeIndex' object is not callable

In [15]:
testf = fs / 'bbc' / 'vtk_df_test.csv'
testVTK_DF = pd.read_csv(testf)
testVTK_DF['subjectid'][0][0:10]

'sub-bbc101'

In [34]:
testVTK_DF.ix[6:, ['subjectid', 'icountgood', 'icounttotal', 'icounttotal' - 'icountgood', 'length']]

TypeError: unsupported operand type(s) for -: 'str' and 'str'

In [16]:
for i, row in testVTK_DF.iterrows():
    print i, row[0][0:10]
    #testVTK_DF.insert(1, 'SUBJID', row[0][0:10], allow_duplicates=True)
#testVTK_DF['SUBJID', 'subjectid']

0 sub-bbc101
1 sub-bbc101
2 sub-bbc105
3 sub-bbc105
4 sub-bbc116
5 sub-bbc106
6 sub-bbc116
7 sub-bbc116
8 sub-bbc118


In [73]:
#selects methods
#testVTK_DF['subjectid'].str[61:88]
#testVTK_DF.loc[:, 'subjectid'].str[61:88]
testVTK_DF['vtk_file_name'].str[61:90]

0    ols_dipy_tensor_medfilt_mori_
1    restore_cam2fsl_tensor_medfil
2    restore_cam2fsl_tensor_medfil
3    wls_fsl_tensor_medfilt_mori_L
4    wls_cam_tensor_medfilt_mori_C
5       wls_cam_tensor_mori_CC.vtk
6    wls_fsl_tensor_medfilt_mori_L
7    wls_fsl_tensor_medfilt_mori_R
8    restore_cam2fsl_tensor_medfil
Name: vtk_file_name, dtype: object

In [70]:
#selects subj id 
#testVTK_DF['Subjid'] = testVTK_DF['subjectid'].str[0:10]

testVTK_DF.ix[:,['Subjid', 'fit_method', 'vtk_file_name', 'FB_reach_dest', 'total_num_FB']]

,Subjid,fit_method,vtk_file_name,FB_reach_dest,total_num_FB
0,sub-bbc101,,sub-bbc101_ses-2_dti_15dir_b1000_1_eddy_corrected_repol_std2_ols_dipy_tensor_medfilt_mori_LeftPostIntCap-35.vtk,0,344
1,sub-bbc101,restore_camino,sub-bbc101_ses-2_dti_15dir_b1000_1_eddy_corrected_repol_std2_restore_cam2fsl_tensor_medfilt_mori_RightPostIntCap-123.vtk,20,245
2,sub-bbc105,restore_camino,sub-bbc105_ses-1_dti_15dir_b1000_1_eddy_corrected_repol_std2_restore_cam2fsl_tensor_medfilt_mori_RightPostIntCap-123.vtk,350,397
3,sub-bbc105,wls_fsl_mf,sub-bbc105_ses-1_dti_15dir_b1000_1_eddy_corrected_repol_std2_wls_fsl_tensor_medfilt_mori_LeftPostIntCap-35.vtk,203,415
4,sub-bbc116,,sub-bbc116_ses-1_dti_15dir_b1000_1_eddy_corrected_repol_std2_wls_cam_tensor_medfilt_mori_CC.vtk,1607,2434
5,sub-bbc106,,sub-bbc106_ses-1_dti_15dir_b1000_1_eddy_corrected_repol_std2_wls_cam_tensor_mori_CC.vtk,2413,3571
6,sub-bbc116,wls_fsl_mf,sub-bbc116_ses-1_dti_15dir_b1000_1_eddy_corrected_repol_std2_wls_fsl_tensor_medfilt_mori_LeftPostIntCap-35.vtk,130,313
7,sub-bbc116,wls_fsl_mf,sub-bbc116_ses-1_dti_15dir_b1000_1_eddy_corrected_repol_std2_wls_fsl_tensor_medfilt_mori_RightPostIntCap-123.vtk,15,101
8,sub-bbc118,restore_camino,sub-bbc118_ses-1_dti_15dir_b1000_1_eddy_corrected_repol_std2_restore_cam2fsl_tensor_medfilt_mori_RightPostIntCap-123.vtk,135,315


In [56]:
#this renames col label 'subjectid' with 'vtk_file_name'
testVTK_DF.rename(columns={'subjectid': 'vtk_file_name', 'icountgood': 'FB_reach_dest', 'icounttotal': 'total_num_FB'}, inplace=True)

In [86]:
testVTK_DF.ix[:,['Subjid', 'fit_method', 'region', 'vtk_file_name', 'FB_reach_dest', 'total_num_FB']]

,Subjid,fit_method,region,vtk_file_name,FB_reach_dest,total_num_FB
0,sub-bbc101,ols_dipy_mf,LeftPostIntCap,sub-bbc101_ses-2_dti_15dir_b1000_1_eddy_corrected_repol_std2_ols_dipy_tensor_medfilt_mori_LeftPostIntCap-35.vtk,0,344
1,sub-bbc101,restore_camino,RightPostIntCap,sub-bbc101_ses-2_dti_15dir_b1000_1_eddy_corrected_repol_std2_restore_cam2fsl_tensor_medfilt_mori_RightPostIntCap-123.vtk,20,245
2,sub-bbc105,restore_camino,RightPostIntCap,sub-bbc105_ses-1_dti_15dir_b1000_1_eddy_corrected_repol_std2_restore_cam2fsl_tensor_medfilt_mori_RightPostIntCap-123.vtk,350,397
3,sub-bbc105,wls_fsl_mf,LeftPostIntCap,sub-bbc105_ses-1_dti_15dir_b1000_1_eddy_corrected_repol_std2_wls_fsl_tensor_medfilt_mori_LeftPostIntCap-35.vtk,203,415
4,sub-bbc116,wls_camino_mf,Corpus_Callosum,sub-bbc116_ses-1_dti_15dir_b1000_1_eddy_corrected_repol_std2_wls_cam_tensor_medfilt_mori_CC.vtk,1607,2434
5,sub-bbc106,wls_camino,Corpus_Callosum,sub-bbc106_ses-1_dti_15dir_b1000_1_eddy_corrected_repol_std2_wls_cam_tensor_mori_CC.vtk,2413,3571
6,sub-bbc116,wls_fsl_mf,LeftPostIntCap,sub-bbc116_ses-1_dti_15dir_b1000_1_eddy_corrected_repol_std2_wls_fsl_tensor_medfilt_mori_LeftPostIntCap-35.vtk,130,313
7,sub-bbc116,wls_fsl_mf,RightPostIntCap,sub-bbc116_ses-1_dti_15dir_b1000_1_eddy_corrected_repol_std2_wls_fsl_tensor_medfilt_mori_RightPostIntCap-123.vtk,15,101
8,sub-bbc118,restore_camino,RightPostIntCap,sub-bbc118_ses-1_dti_15dir_b1000_1_eddy_corrected_repol_std2_restore_cam2fsl_tensor_medfilt_mori_RightPostIntCap-123.vtk,135,315


In [85]:
#this renames col label 'subjectid' with 'vtk_file_name'
testVTK_DF.rename(columns={'subjectid': 'vtk_file_name', 'icountgood': 'FB_reach_dest', 'icounttotal': 'total_num_FB'}, inplace=True)
#makes subj id fm 1st 10 chars of vtk_file_name
testVTK_DF['Subjid'] = testVTK_DF['vtk_file_name'].str[0:10]
#makes fit methods
testVTK_DF.loc[testVTK_DF['vtk_file_name'].str.contains('restore_cam2fsl_tensor_medf'), 'fit_method'] = 'restore_camino'
testVTK_DF.loc[testVTK_DF['vtk_file_name'].str.contains('wls_fsl_tensor_medf'), 'fit_method'] = 'wls_fsl_mf'
testVTK_DF.loc[testVTK_DF['vtk_file_name'].str.contains('ols_dipy_tensor_med'), 'fit_method'] = 'ols_dipy_mf'
testVTK_DF.loc[testVTK_DF['vtk_file_name'].str.contains('wls_cam_tensor_medf'), 'fit_method'] = 'wls_camino_mf'
testVTK_DF.loc[testVTK_DF['vtk_file_name'].str.contains('wls_cam_tensor_mori'), 'fit_method'] = 'wls_camino'
#makes region names for regions col.
testVTK_DF.loc[testVTK_DF['vtk_file_name'].str.contains('LeftPostIntCap'), 'region'] = 'LeftPostIntCap'
testVTK_DF.loc[testVTK_DF['vtk_file_name'].str.contains('RightPostIntCap'), 'region'] = 'RightPostIntCap'
testVTK_DF.loc[testVTK_DF['vtk_file_name'].str.contains('mori_CC'), 'region'] = 'Corpus_Callosum'

In [17]:
pd.get_option("display.max_rows")

60